In [10]:
'''
BT17CSE030
Hardik Gandhi

DPLL ALGO 


'''

'\nBT17CSE030\nHardik Gandhi\n\nDPLL ALGO \n\n\n'

In [11]:
# Library imports
import os

In [12]:
def pre_process(clause_list, proposition_count, unit_clause):

    flag = True

    while(flag and len(clause_list)!=0):
        flag = False
        for key in proposition_count:
            if ((proposition_count[key][0]==0 and proposition_count[key][1]!=0) or (proposition_count[key][0]!=0 and proposition_count[key][1]==0)):
                flag = True
            
            if (flag == True and len(clause_list)!=0):
                clause_key_list = list(clause_list.keys())      

                for clause_key in clause_key_list:
                    clause = clause_list[clause_key]
                    sign = clause.get(key, None)
                    if sign is not None:
                        
                        '''Removing pure litters'''
                        clause_list.pop(clause_key)
                        
                        '''Updating the count of occurrence of propositional variables'''
                        for k in clause:
                            v, s = k, clause[k]
                            l = proposition_count[v]

                            if s == 1:
                                l[0] = l[0] - 1
                            else:
                                l[1] = l[1] - 1

                            proposition_count[v] = l

                        '''Removing dangling unit clause'''
                        if len(clause) == 1:
                            try:
                                unit_clause.remove(clause_key)
                            except:
                                pass

In [13]:
def make_clause(clause, line, count_dict, line_no, unit_clause):
    '''
    Storing the propositional clauses
    '''
    
    proposition = line.split()
    new_clause = dict()

    for p in proposition:
        if p != '0':
            elem, sign = None, None  
            if p[0] == '-':
                elem = p[1:]
                sign = -1
            else:
                elem = p
                sign = 1

            new_clause[elem] = sign

            '''
            Incrementing count of propositional variable
            '''
            if count_dict.get(elem, None) is None:
                count_dict[elem] = [0, 0]

            count_list = count_dict[elem]
            if sign == 1:
                count_list[0] = count_list[0] + 1
            else:
                count_list[1] = count_list[1] + 1

            count_dict[elem] = count_list

    clause[line_no] = new_clause

    '''
    Chaecking whether unit clause or not
    '''
    if len(proposition) == 2:
        unit_clause.append(line_no)
        

In [14]:
def assign_value(clause_list, proposition_count, unit_clause, var, assign, deletion, removal):
    '''
    Assigning the value to a propositional variable and perform changes in clauses
    and keeping the record of doing that
    '''
    
    ret = 0
    '''Assigning and updating'''
    clause_key_list = list(clause_list.keys())

    for clause_key in clause_key_list:
        clause = clause_list[clause_key]

        sign = clause.get(var, None)
        if sign is not None:
            '''Same sense'''
            if sign == assign:
                print("Delete clause #" + str(clause_key))
                clause_list.pop(clause_key)
                '''Keeping the record of the clause being deleted'''
                deletion[clause_key] = clause
                '''Updating the count of occurrence of propositional variables'''
                for k in clause:
                    v, s = k, clause[k]
                    l = proposition_count[v]

                    if s == 1:
                        l[0] = l[0] - 1
                    else:
                        l[1] = l[1] - 1

                    proposition_count[v] = l

                '''Removing dangling unit clause'''
                if len(clause) == 1:
                    try:
                        unit_clause.remove(clause_key)
                    except:
                        pass

            elif (sign + assign) == 0:
                '''Opposite sense'''
                '''Deleting the unnecessary variable from the clause and keeping that record'''
                print("Remove var '" + var + "' from clause #" + str(clause_key))
                clause.pop(var)
                clause_list[clause_key] = clause
                removal.append(clause_key)

                l = proposition_count[var]
                if sign == 1:
                    l[0] = l[0] - 1
                else:
                    l[1] = l[1] - 1

                proposition_count[var] = l

                '''Checking whether any unit clause is created'''
                if len(clause) == 1:
                    unit_clause.append(clause_key)
                    ret =ret+1

    return ret

In [15]:
def revert_assignment(clause_list, proposition_count, unit_clause, var, assign, deletion, removal, unit):
    '''
    Reverting back whatever was assigned in the current stage
    '''
    
    
    for del_key in deletion:
        print("Adding clause #" + str(del_key))
        clause = deletion[del_key]

        '''Adding the deleted clauses back anf updating the count of occurrence of variables'''
        clause_list[del_key] = clause
        for k in clause:
            v, s = k, clause[k]
            l = proposition_count[v]

            if s == 1:
                l[0] = l[0] + 1
            else:
                l[1] = l[1] + 1

            proposition_count[v] = l

        if len(clause) == 1:
            unit_clause.append(del_key)

    '''Adding the deleted variable back to their clauses and also updating their count'''
    sign = (-1)*assign
    for clause_key in removal:
        print("Adding var '" + var + "' to clause #" + str(clause_key))
        clause_list[clause_key][var] = sign

        l = proposition_count[var]
        if sign == 1:
            l[0] = l[0] + 1
        else:
            l[1] = l[1] + 1

        proposition_count[var] = l

    '''If any unit clause was created, remove it'''
    if unit >0:
        while(unit>0):
            unit_clause.pop(-1)
            unit=unit-1

In [16]:
def dpll(clause_list, proposition_count, unit_clause,solution):
    '''
    dpll algorithm
    return True if clause_list becomes True for certain combination of assignments to variables
    '''
    
    if len(clause_list) == 0:
        return True

    elif len(list(filter(lambda x: x==0, [len(clause_list[key]) for key in clause_list]))) != 0:
        return False

    else:
        var, assign = None, None
        '''
        Evaluating which variable should we assign a value and what value via heuristic
        '''
        flag=0
        flag1=0
        unit_count=0
#         unit clause heurestics
        if len(unit_clause) != 0:
            el = unit_clause.pop(0)

            var = list(clause_list[el].keys())[0]
            assign = clause_list[el][var]
            flag=1
            if assign == 1 :
                solution[var]=True
            else:
                solution[var]=False
#             pure literal
        elif flag==0:
            for key in proposition_count:
                if proposition_count[key][0]==0 and proposition_count[key][1]!=0 and flag1==0:
                    var=key
                    assign=-1
                    flag1=1
                    
                elif proposition_count[key][0]!=0 and proposition_count[key][1]==0 and flag1==0:
                    var=key
                    assign=1
                    flag1=1
                    
            if flag1==1:
                flag=1
                if assign == 1 :
                    solution[var]=True
                else:
                    solution[var]=False
                    
#             max occurance
                
        if flag==0:
            p_l = [(key, proposition_count[key][0], proposition_count[key][1]) for key in proposition_count]
            p_l = sorted(p_l, key=lambda x: x[1]+x[2], reverse=True)

            pick = p_l[0]
            var = pick[0]
            assign = 1 if pick[1] >= pick[2] else -1
            if assign == 1 :
                solution[var]=True
            else:
                solution[var]=False

        '''
        Assigning the value
        '''
        print("\n--------------------------------------\n")
        print("Assigning '" + var + "' = " + str(assign) + " : \n")
        deletion, removal, unit = {}, [], 0
        unit = assign_value(clause_list, proposition_count, unit_clause, var, assign, deletion, removal)
        
        '''Preceeding to next phase'''
        effect = dpll(clause_list, proposition_count, unit_clause, solution)

        if effect == True:
            '''Assignment has been found for which dpll becomes True...Job Done'''
            return True

        else:
            '''Dpll becomes false for current assignment combination...Try another'''
            print("\n--------------------------------------\n")
            print("Result False, So, Reverting '" + var + "' = " + str(assign) + " : \n")
            revert_assignment(clause_list, proposition_count, unit_clause, var, assign, deletion, removal, unit)
            '''Changing the value assigned'''
            assign = (-1)*assign
            if assign == 1 :
                    solution[var]=True
            else:
                solution[var]=False

            deletion, removal, unit = {}, [], 0
            '''Assigningment with new combination'''
            print("\n--------------------------------------\n")
            print("Assigning '" + var + "' = " + str(assign) + " : \n")
            unit = assign_value(clause_list, proposition_count, unit_clause, var, assign, deletion, removal)
            '''Preceeding to next phase'''
            effect_1 = dpll(clause_list, proposition_count, unit_clause, solution)

            if effect_1 == True:
                '''Assignment has been found for which dpll becomes True...Job Done'''
                return True

            else:
                '''Dpll becomes false for current assignment combination...Try another'''
                print("\n--------------------------------------\n")
                print("Result False, So, Reverting to previous state '" + var + "' = " + str(assign) + " : \n")
                revert_assignment(clause_list, proposition_count, unit_clause, var, assign, deletion, removal, unit)
                print("\n--------------------------------------\n")
                print("NO AVAILABLE OPTIONS, BACKTRACK")
                return False


In [17]:
def main():
#     Var intailize
    clause_list = {}
    proposition_count = {}
    unit_clause = []
    solution={}
    
#     File read
    dirname = os.getcwd()
    filename = os.path.join(dirname,'dpll_input.txt')
    with open(filename,'r') as f:
        lines = f.readlines()
        line_no = 0
        for line in lines:
            make_clause(clause_list, line, proposition_count, line_no, unit_clause)
            line_no = line_no + 1

    print("\nQuestion : \n")

    print("CLAUSES : ", clause_list)
    

    print()
    print("Solution : ")
    
    '''
    DPLL
    '''
    
    pre_process(clause_list, proposition_count, unit_clause)
    
    result = dpll(clause_list, proposition_count, unit_clause, solution)

    print("\n--------------------------------------\n")
    if result:
        print("Result is = " + str(result) + " \n")
        print("------------------------------------------------------------------------------------------")
        print("Final Solution is:")
        print(solution)
    else:
        print("Result is = " + str(result) + " \n")

In [18]:
main()


Question : 

CLAUSES :  {0: {'16': 1, '17': 1, '30': 1}, 1: {'17': -1, '22': 1, '30': 1}, 2: {'17': -1, '22': -1, '30': 1}, 3: {'16': 1, '30': -1, '47': 1}, 4: {'16': 1, '30': -1, '47': -1}, 5: {'16': -1, '21': -1, '31': 1}, 6: {'16': -1, '21': -1, '31': -1}, 7: {'16': -1, '21': 1, '28': -1}, 8: {'13': -1, '21': 1, '28': 1}, 9: {'13': 1, '16': -1, '18': 1}, 10: {'13': 1, '18': -1, '38': -1}, 11: {'13': 1, '18': -1, '31': -1}, 12: {'31': 1, '38': 1, '44': 1}, 13: {'8': -1, '31': 1, '44': -1}, 14: {'8': 1, '12': -1, '44': -1}, 15: {'8': 1, '12': 1, '27': -1}, 16: {'12': 1, '27': 1, '40': 1}, 17: {'4': -1, '27': 1, '40': -1}, 18: {'12': 1, '23': 1, '40': -1}, 19: {'3': -1, '4': 1, '23': -1}, 20: {'3': 1, '23': -1, '49': -1}, 21: {'3': 1, '13': -1, '49': -1}, 22: {'23': -1, '26': -1, '49': 1}, 23: {'12': 1, '34': -1, '49': 1}, 24: {'12': -1, '26': 1, '34': -1}, 25: {'19': 1, '34': 1, '36': 1}, 26: {'19': -1, '26': 1, '36': 1}, 27: {'30': -1, '34': 1, '36': -1}, 28: {'24': 1, '34': 1, '36'

Remove var '16' from clause #5
Remove var '16' from clause #6
Remove var '16' from clause #7
Remove var '16' from clause #9

--------------------------------------

Assigning '13' = -1 : 

Remove var '13' from clause #10
Remove var '13' from clause #11
Delete clause #21
Delete clause #78
Delete clause #79
Delete clause #8
Remove var '13' from clause #9

--------------------------------------

Assigning '18' = 1 : 

Remove var '18' from clause #10
Remove var '18' from clause #11
Delete clause #74
Delete clause #9

--------------------------------------

Assigning '38' = -1 : 

Delete clause #10
Remove var '38' from clause #12
Remove var '38' from clause #67
Delete clause #76

--------------------------------------

Assigning '31' = -1 : 

Delete clause #11
Remove var '31' from clause #12
Remove var '31' from clause #13
Delete clause #66
Remove var '31' from clause #5
Delete clause #6

--------------------------------------

Assigning '44' = 1 : 

Delete clause #12
Remove var '44' from c